<a href="https://colab.research.google.com/github/Kn3ule/Text-Mining-Sentiment-Analysis-for-player-scouting/blob/main/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from dotenv import load_dotenv
import os
import json
import random
from bs4 import BeautifulSoup as bs
import re
import random

# Instagram

In [8]:
def get_post_info(driver, link):
    driver.get(f'{link}?img_index=1')  # fully load a moderate dynanmic webpage
    wait = WebDriverWait(driver, 10)
    # **Schritt 1: Button klicken, um Kommentare auf "Neueste" umzustellen**

    # **Schritt 1: Klicken auf den Span mit dem Text "Für dich"**

    try:
        # Finden des Span-Elements mit dem Text "Für dich"
        fuer_dich_span = driver.find_element(By.XPATH, "//span[text()='Für dich']")
        fuer_dich_span.click()
        time.sleep(1)  # Warten, bis das Menü erscheint

        # **Schritt 2: Auswählen des Span mit dem Text "Neueste"**

        neueste_span = driver.find_element(By.XPATH, "//span[text()='Neueste']")
        neueste_span.click()
        time.sleep(2)  # Warten, bis die Kommentare neu geladen sind

    except Exception as e:
        print(f'Fehler beim Umschalten auf "Neueste" Kommentare: {e}')
    
    res = driver.page_source
    soup = bs(res,'html.parser')
    body = soup.find('body')
    try:
        captions = body.find_all('span', {'class': 'x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs xt0psk2 x1i0vuye xvs91rp xo1l8bm x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj'})
        for caption in captions:
            if caption.text != "und":
                caption = caption.text
    except:
        caption = ''
        print("Keine Caption gefunden")

    # **Schritt 3: Scrollen in der Kommentarspalte, um alle Kommentare zu laden**

    comments_section = driver.find_element(By.CSS_SELECTOR, '.x5yr21d.xw2csxc.x1odjw0f.x1n2onr6')
    #comments_section = driver.find_element(By.CSS_SELECTOR, '.x78zum5.xdt5ytf.x1iyjqo2.xs83m0k.x2lwn1j.x1odjw0f.x1n2onr6.x9ek82g.x6ikm8r.xdj266r.x11i5rnm.x4ii5y1.x1mh8g0r.xexx8yu.x1pi30zi.x18d9i69.x1swvt13')
    last_height = driver.execute_script("return arguments[0].scrollHeight", comments_section)

    while True:
        try:
            # Scrollen nach unten in der Kommentare-Sektion
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", comments_section)
            time.sleep(5)  # Warten, bis neue Kommentare geladen sind

            # Überprüfen, ob neue Kommentare geladen wurden
            new_height = driver.execute_script("return arguments[0].scrollHeight", comments_section)
            if new_height == last_height:
                break  # Alle Kommentare sind geladen
            last_height = new_height
        except:
            print("scrollen durch Kommentare nicht möglich")

    # **Schritt 4: Extrahieren der Kommentare**

    time.sleep(5)
    
    res = driver.page_source
    soup = bs(res,'html.parser')
    body = soup.find('body')  #  creating a beautifulsoup object using source page variable created above
    divs = body.find_all( 'div', {'class': 'x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1cy8zhl x1oa3qoh x1nhvcw1'}) # finding all div tags with a class attribute from beautifulsoup object

    # iterating through all div tags and getting span tag text using for loop
    comment_insta=[]
    for div in divs[0:]:       # iterating through all div tags using for loop
        spans = div.find_all('span') # finding span tags which are within div tag  with a class attribute
        
        for span in spans:            # iterating over spans variable to get requuired text
            comment_insta.append(span.text)  # saving cleaned comments in comments list

    # Erstellen eines Dictionaries für den aktuellen Post
    post_data = {
        'link': link,
        'caption': caption,
        'comments': comment_insta
    }

    return post_data

In [15]:
def get_links_of_posts(driver, club):
    links = []

    # Instagram öffnen
    driver.get("https://www.instagram.com")
    time.sleep(3)

    # Cookies aus der JSON-Datei laden
    with open('../cookies/insta-cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)

    # Cookies in den Browser laden
    for cookie in cookies:
        # Passe das Cookie an, falls es Probleme gibt (z. B. Domain anpassen)
        if 'sameSite' in cookie:
            del cookie['sameSite']  # Selenium unterstützt nicht alle Cookie-Attribute
        driver.add_cookie(cookie)

    # Seite aktualisieren, um die Cookies anzuwenden und als eingeloggter Nutzer zu interagieren
    driver.get("https://www.instagram.com")
    time.sleep(3)

    # Optional: Wartezeit erhöhen, um sicherzustellen, dass die Seite vollständig geladen ist
    time.sleep(5)

    # "Benachrichtigungen aktivieren" umgehen
    '''
    try:
        # Das Element "Jetzt nicht" suchen und klicken
        not_now_button = driver.find_element(By.XPATH, "//button[text()='Jetzt nicht']")
        not_now_button.click()
        print("Benachrichtigungen wurden deaktiviert.")
        time.sleep(2)
    except Exception as e:
        print("Kein Benachrichtigungs-Dialog oder Fehler beim Klicken:", e)
    '''
    
    # Weiter mit dem Scraping-Prozess, z.B. zum Vereinsprofil navigieren
    driver.get(f'https://www.instagram.com/{club}/')
    time.sleep(5)

                
    for i in range(20): #Je öfter gescrollt wird, desto mehr posts werden generiert (x=20, ca. 120 Posts)
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Finden aller Reihen
            #rows = driver.find_elements(By.CSS_SELECTOR, '._ac7v.xras4av.xgc1b0m.xat24cr.xzboxd6')  # Passen Sie den Selektor an
            rows = driver.find_elements(By.CSS_SELECTOR, '._ac7v.x1f01sob.xcghwft.xat24cr.xzboxd6')
            

            for row in rows:
                try:
                    # Finden aller Kacheln innerhalb der Reihe
                    tiles = row.find_elements(By.CSS_SELECTOR, '.x1lliihq.x1n2onr6.xh8yej3.x4gyw5p.x1ntc13c.x9i3mqj.x11i5rnm.x2pgyrj')
                    
                    for tile in tiles:
                        link_element = tile.find_element(By.TAG_NAME, 'a')
                        href = link_element.get_attribute('href')
                        if href:
                            links.append(href)
                except:
                    (print("konnte nicht verarbeitet werden"))
        except:
            print("Scrollen durch Posts nicht möglich")
        
        time.sleep(5)

    return links




In [10]:
def get_posts_and_comments_by_club(club):
    #Scrape Instagram Post and Comments
    driver = webdriver.Chrome()
    posts = []

    links = get_links_of_posts(driver=driver, club=club)
    
    # Annahme: links ist eine Liste von Links, die durchlaufen werden sollen
    for idx, link in enumerate(list(dict.fromkeys(links)), start=1):
        post = get_post_info(driver=driver, link=link)
        posts.append(post)

        # Normale Wartezeit zwischen den Schleifendurchläufen
        time.sleep(random.randint(7, 23)) 

        # Überprüfen, ob die Iteration ein Vielfaches von 10 ist
        if idx % 10 == 0:
            print(f"Iteration {idx}: 2 Minuten Pause.")
            time.sleep(120)  # 120 Sekunden (2 Minuten) Pause
    
    driver.quit()
    
    return posts


In [ ]:
#Scrape Instagram Post and Comments
driver = webdriver.Chrome()
posts = []
club = 'palmeiras'
links = get_links_of_posts(driver=driver, club=club)
print(links)
    
# Annahme: links ist eine Liste von Links, die durchlaufen werden sollen
for idx, link in enumerate(list(dict.fromkeys(links)), start=1):
    # URL in Teile aufsplitten
    parts = link.split("/")

    # Zusammensetzen ohne den dynamischen Teil (indem du das 4. Element weglässt)
    clean_link = "/".join(parts[:3] + parts[4:])
    try:
        post = get_post_info(driver=driver, link=clean_link)
        posts.append(post)
    except:
        print('Konnte nicht geöffnet werden')

        # Normale Wartezeit zwischen den Schleifendurchläufen
    time.sleep(random.randint(7, 23)) 

    # Überprüfen, ob die Iteration ein Vielfaches von 10 ist
    if idx % 10 == 0:
        print(f"Iteration {idx}: X Minuten Pause.")
        time.sleep(random.randint(60, 180)) # 120 Sekunden (2 Minuten) Pause
    
driver.quit()

# Convert and write JSON object to file
with open("../data/instagram/palmeiras_instacomments.json", "w", encoding='utf-8') as outfile: 
    json.dump(posts, outfile, ensure_ascii=False)

konnte nicht verarbeitet werden
konnte nicht verarbeitet werden
konnte nicht verarbeitet werden
konnte nicht verarbeitet werden
['https://www.instagram.com/palmeiras/reel/DG4N3W7ulbP/', 'https://www.instagram.com/palmeiras/p/DGtQKuFCYdb/', 'https://www.instagram.com/palmeiras/reel/DGCIpG3u5w7/', 'https://www.instagram.com/avantipalmeiras/reel/DHBhJwAs1Im/', 'https://www.instagram.com/palmeiras/p/DHAMIqBNRxf/', 'https://www.instagram.com/palmeiras/reel/DG_69p_NmWV/', 'https://www.instagram.com/palmeiras/p/DG_2lLPRYlt/', 'https://www.instagram.com/palmeiras/p/DG_y0XrPWnD/', 'https://www.instagram.com/palmeiras/p/DG_i5HAPSuh/', 'https://www.instagram.com/palmeiras/reel/DG_eItwS2EF/', 'https://www.instagram.com/palmeiras/p/DG_cCFcNXPU/', 'https://www.instagram.com/palmeiras/reel/DG_VM8txFfH/', 'https://www.instagram.com/palmeiras/reel/DG4N3W7ulbP/', 'https://www.instagram.com/palmeiras/p/DGtQKuFCYdb/', 'https://www.instagram.com/palmeiras/reel/DGCIpG3u5w7/', 'https://www.instagram.com/avan

In [21]:
# Convert and write JSON object to file
with open("../data/instagram/vfb2_instacomments.json", "w", encoding='utf-8') as outfile: 
    json.dump(posts, outfile, ensure_ascii=False)

In [38]:
#Scrape Instagram Post and Comments
driver = webdriver.Chrome()
link = "https://www.instagram.com/p/DBI2Xb_psct/"
# Instagram öffnen
driver.get("https://www.instagram.com")
time.sleep(3)

# Cookies aus der JSON-Datei laden
with open('cookies/insta-cookies.json', 'r') as cookies_file:
    cookies = json.load(cookies_file)

# Cookies in den Browser laden
for cookie in cookies:
    # Passe das Cookie an, falls es Probleme gibt (z. B. Domain anpassen)
    if 'sameSite' in cookie:
        del cookie['sameSite']  # Selenium unterstützt nicht alle Cookie-Attribute
    driver.add_cookie(cookie)

# Seite aktualisieren, um die Cookies anzuwenden und als eingeloggter Nutzer zu interagieren
driver.get("https://www.instagram.com")
time.sleep(3)

# Optional: Wartezeit erhöhen, um sicherzustellen, dass die Seite vollständig geladen ist
time.sleep(5)

# "Benachrichtigungen aktivieren" umgehen
try:
    # Das Element "Jetzt nicht" suchen und klicken
    not_now_button = driver.find_element(By.XPATH, "//button[text()='Jetzt nicht']")
    not_now_button.click()
    print("Benachrichtigungen wurden deaktiviert.")
    time.sleep(2)
except Exception as e:
    print("Kein Benachrichtigungs-Dialog oder Fehler beim Klicken:", e)


post = get_post_info(driver=driver, link=link)

Benachrichtigungen wurden deaktiviert.


In [39]:
print(post)

{'link': 'https://www.instagram.com/p/DBI2Xb_psct/', 'caption': '"He\'s from Barrancas, and he plays for Liverpool..." 🎵🇨🇴Luis Diaz on how his upbringing made him the footballer he is 👊 Read the full article on our website. Link in the bio 🔗', 'comments': ['❤️', 'Our lad from Porto 👏🔥', 'Does somebody have 2 tickets for the game in Leipzig to sell?', 'He comes to score😂', '😍😍', '🔥🔥🔥🔥🔥', 'I love him lucho ❤️', '🔥🔥🔥', '🔥🔥🔥', '🔥🔥', '❤️❤️', 'Mi Colombiano favorito 🔥🔥🔥❤️', 'Admin please edit this and add the song. It’s incomplete 😂', 'Double his wages , guys on 🔥', 'We call him lucho, he came from porto, he came to score came to score came to score. . . score. . . score. . .', 'Cook Chelsea 😈', '🔥🔥🔥🔥', '...he came to score, score, score 🎼 🎶 🎵', 'Me gustas tu Luiz Diaz👏👏👏', 'We call him Lucho 🔥🤌', '🔋❤️', 'Na na na na na🎵🎵', 'Ja la la la la ..ja la la la la 🔥🙌', 'absence', '❤️❤️❤️', 'Luis diaz button❤️🔥>>>>>>>>>', 'Admin forgot to play the song 😂', '🇨🇴🇨🇴']}


In [33]:
import json

#posts = get_posts_and_comments_by_club(club='chelseafc')
# Convert and write JSON object to file
with open("data/instagram/lille_instacomments.json", "w", encoding='utf-8') as outfile: 
    json.dump(posts, outfile, ensure_ascii=False)

In [299]:
print(len(posts))

12


# Twitter

In [12]:
def login_with_cookies(driver):
    # Twitter öffnen
    driver.get("https://x.com/")
    time.sleep(3)

    # Cookies aus der JSON-Datei laden
    with open('cookies/twitter-cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)

    # Cookies in den Browser laden
    for cookie in cookies:
        # Passe das Cookie an, falls es Probleme gibt (z. B. Domain anpassen)
        if 'sameSite' in cookie:
            del cookie['sameSite']  # Selenium unterstützt nicht alle Cookie-Attribute
        driver.add_cookie(cookie)

    # Seite aktualisieren, um die Cookies anzuwenden und als eingeloggter Nutzer zu interagieren
    driver.get("https://x.com")
    time.sleep(3)

In [13]:
def get_comments_by_club(driver, club):

    #Vereinsprofil öffnen
    driver.get(f"https://x.com/{club}")
    time.sleep(3)

    


In [23]:
def get_tweets_by_hashtag(driver, hashtag):

    #Tweets mit Hashtag laden
    driver.get(f"https://x.com/search?q=%23{hashtag}&src=typed_query&f=live")
    time.sleep(3)
    tweet_data = []

    # Scrollen, um mehr Tweets zu laden
    for i in range(25):  # Passe die Anzahl der Scrolls nach Bedarf an

        # Warte, bis die Tweets geladen sind (alle article-Elemente mit role='article')
        wait = WebDriverWait(driver, 10)
        tweets = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@role="article"]')))


        # Schleife durch die gefundenen Tweets und extrahiere den Text
        for tweet in tweets:
            try:
                tweet_text = tweet.find_element(By.XPATH, './/div[@lang]').text  # Der Haupttext eines Tweets
                tweet_data.append(tweet_text.replace('\n', ''))
            except Exception as e:
                print("Fehler beim Extrahieren eines Tweets:", e)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.randint(4, 11)) # putting some sleep time between every loop

    data = {
        'hashtag': hashtag,
        'tweets': tweet_data
    }
    return data



In [28]:
def get_tweets_by_name(driver, name):
    player_name = name.replace(' ', '%20')
    #Tweets mit Hashtag laden
    driver.get(f'https://x.com/search?q="{player_name}"&src=typed_query&f=live')
    time.sleep(3)
    tweet_data = []

    # Scrollen, um mehr Tweets zu laden
    for i in range(25):  # Passe die Anzahl der Scrolls nach Bedarf an

        # Warte, bis die Tweets geladen sind (alle article-Elemente mit role='article')
        wait = WebDriverWait(driver, 10)
        tweets = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@role="article"]')))


        # Schleife durch die gefundenen Tweets und extrahiere den Text
        for tweet in tweets:
            try:
                tweet_text = tweet.find_element(By.XPATH, './/div[@lang]').text  # Der Haupttext eines Tweets
                tweet_data.append(tweet_text.replace('\n', ''))
            except Exception as e:
                print("Fehler beim Extrahieren eines Tweets:", e)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.randint(4, 11)) # putting some sleep time between every loop

    data = {
        'name': name,
        'tweets': tweet_data
    }
    return data

In [24]:
driver = webdriver.Chrome()

login_with_cookies(driver=driver)
#get_comments_by_club(driver=driver, club='VfB')
tweets = get_tweets_by_hashtag(driver=driver, hashtag='CFC')

driver.quit()

In [29]:
driver = webdriver.Chrome()

login_with_cookies(driver=driver)
#get_comments_by_club(driver=driver, club='VfB')
tweets_by_name = get_tweets_by_name(driver=driver, name='Cole Palmer')

driver.quit()

Fehler beim Extrahieren eines Tweets: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[@lang]"}
  (Session info: chrome=129.0.6668.72); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102f6bed4 cxxbridge1$str$ptr + 1906348
1   chromedriver                        0x0000000102f64344 cxxbridge1$str$ptr + 1874716
2   chromedriver                        0x0000000102b78264 cxxbridge1$string$len + 89492
3   chromedriver                        0x0000000102bbc514 cxxbridge1$string$len + 368708
4   chromedriver                        0x0000000102bb2ad8 cxxbridge1$string$len + 329224
5   chromedriver                        0x0000000102bf67d4 cxxbridge1$string$len + 606980
6   chromedriver                        0x0000000102bb1134 cxxbridge1$string$len + 322660
7   chromedriver                        0x

In [269]:
print(tweets_by_name)

{'name': 'Jamie Leweling', 'tweets': ['Wie Jamie Leweling auf der Außenbahn überzeugt #VfB #Statistik', ' Jamie Leweling vs Real Madrid was simply brilliant last week!A player to watch : @JxckzLFC', 'Jamie Leweling vs Real Madrid 17/9/2024', 'Jamie Leweling: "Wir waren eine Topmannschaft, wir sind eine Topmannschaft und wir werden weiter eine sein."', 'Jamie Leweling against Dortmund90mins played Touches 51Recoveries 6Passes into final third 4Accurate Passes 24/29(83%)Accurate crosses 3/5(60%)Bridget Otoo|Akuffo Addo|Thomas Partey|Xavi', 'Jamie #Leweling hat beim #VfB den nächsten Entwicklungsschritt gemacht.', 'Jamie wird links spielen', '[VIDEO] – BLACK STARS  COACH OTTO ADDO TURNS ATTENTION TO JAMIE LEWELING WHO ASSISTED A GOAL VS REAL MADRID', 'Wieviele Tore macht jamie leweling heute für mein kickbase team', "VfB Stuttgart coach Sebastian HoeneÃ\x9f praises Ghanaian winger Jamie Leweling's form http://dlvr.it/TDVBVZ", 'jamie leweling', 'Mein #VfB MOTM im Spiel #RMAVfB Jamie Leweli

In [30]:
import json

# Convert and write JSON object to file
with open("data/twitter/palmer_tweets.json", "w", encoding='utf-8') as outfile: 
    json.dump(tweets_by_name, outfile, ensure_ascii=False)